In [35]:
import pandas as pd
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

diabetes = pd.read_csv('data_14/diabetes_data.csv')
diabetes.head()
diabetes.shape

(778, 10)

1. Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

In [38]:
duplicates = diabetes[diabetes.duplicated()]
print(duplicates.shape[0], 'дублей')
diabetes = diabetes.drop_duplicates()
print('Число строк', diabetes.shape[0])

10 дублей
Число строк 768


2. Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.99: удалите все признаки, для которых 99 % значений повторяются или 99 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).

In [39]:
low_information_cols = [] #список неинформативных признаков

for col in diabetes.columns: #цикл по всем столбцам
    top_freq = diabetes[col].value_counts(normalize=True).max() #наибольшая относительная частота в признаке
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count() #доля уникальных значений от размера признака
    if top_freq > 0.99: # сравниваем наибольшую частоту с порогом
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    if nunique_ratio > 0.99: # сравниваем долю уникальных значений с порогом
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
diabetes.drop(['Gender'], axis=1, inplace=True)
diabetes.shape

KeyError: "['Gender'] not found in axis"

In [40]:
diabetes.shape

(768, 9)

3. Попробуйте найти пропуски в данных с помощью метода isnull().
Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!
Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.
Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

In [41]:
nul_cols_persent = diabetes.isnull().mean()*100
nul_cols = nul_cols_persent[nul_cols_persent > 0]
print(nul_cols)

Series([], dtype: float64)


In [42]:
diabetes['Glucose'] = diabetes['Glucose'].apply(lambda x: np.nan if x==0 else x)
diabetes['BloodPressure'] = diabetes['BloodPressure'].apply(lambda x: np.nan if x==0 else x)
diabetes['SkinThickness'] = diabetes['SkinThickness'].apply(lambda x: np.nan if x==0 else x)
diabetes['Insulin'] = diabetes['Insulin'].apply(lambda x: np.nan if x==0 else x)
diabetes['BMI'] = diabetes['BMI'].apply(lambda x: np.nan if x==0 else x)

diabetes.isnull().mean().round(2).sort_values(ascending=False)
        

Insulin                     0.49
SkinThickness               0.30
BloodPressure               0.05
Glucose                     0.01
BMI                         0.01
Pregnancies                 0.00
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64

4. Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

In [43]:
thresh = diabetes.shape[0]*0.7
diabetes = diabetes.dropna(
    how='any',
    thresh=thresh,
    axis=1
)
diabetes.shape

(768, 8)

5. Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

In [44]:
m = diabetes.shape[1]
diabetes = diabetes.dropna(thresh=m-2, axis=0)
print(diabetes.shape[0])

761


6. В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.

In [46]:
null_data = diabetes.isnull().sum()
cols = null_data[null_data>0].index

for col in cols:
    diabetes[col] = diabetes[col].fillna(diabetes[col].median())

print(diabetes['SkinThickness'].mean().round(1))

29.1


7. Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

In [49]:
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr(diabetes, 'SkinThickness')
print(outliers.shape[0])

87


8. Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

In [51]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score(diabetes, 'SkinThickness')
print(outliers.shape[0])

4


9. На приведённой гистограмме показано распределение признака DiabetesPedigreeFunction. Такой вид распределения очень похож на логнормальный, и он заставляет задуматься о логарифмировании признака. Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.
Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!). Какова разница между двумя этими числами (вычтите из первого второе)?

In [53]:
outliers, cleaned = outliers_iqr(diabetes, 'DiabetesPedigreeFunction')
print(outliers.shape[0])

def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(diabetes, 'DiabetesPedigreeFunction', log_scale=True)
print(outliers.shape[0])

29
0
